# Preparing Data for Graph Construction

As part of our larger data science goal for this workshop, we will be working with data reflecting the entire road network of Great Britain. We have as a starting point road data extracted into tabular csv format from official [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files. Ultimately, we would like to use cuGraph to perform GPU-accelerated graph analytics on this data, but in order to do so, we need to do some preprocessing to get it ready for graph creation.

In this notebook you will be learning additional cuDF data transformation techniques, including the use of merges, in a demonstration of prepping the initial data for ingestion by cuGraph. Next, you will do a series of exercises to perform a similar transformation of the data for the creation of a graph with different edge weights.

## Objectives

By the time you complete this notebook you will be able to:

- Perform GPU-accelerated merge operations with cuDF
- Prepare data for cuGraph ingest
- Create a GPU-accelerated graph

## Imports

In addition to `cudf`, for this notebook we will also import `cugraph`, which we will use (after data preparation) to construct a GPU-accelerated graph. We also import `networkx` for a brief performance comparison later on.

In [ ]:
import cudf
import cugraph as cg

import networkx as nx

## Read Data

In this notebook we will be working with two data sources that will help us create a graph of the UK's road networks.

### UK Road Nodes

The first data table describes the nodes in the road network: endpoints, junctions (including roundabouts), and points that break up a long stretch of curving road so that it can be mapped correctly (instead of as a straight line).

The coordinates for each point are in the OSGB36 format we explored earlier in section 1-05.

In [ ]:
road_nodes = cudf.read_csv('./data/road_nodes_1-06.csv')
road_nodes.head()

In [ ]:
road_nodes.dtypes

In [ ]:
road_nodes.shape

In [ ]:
road_nodes['type'].unique()

### UK Road Edges

The second data table describes road segments, including their start and end points, how long they are, and what kind of road they are.

In [ ]:
road_edges = cudf.read_csv('./data/road_edges_1-06.csv')
road_edges.head()

In [ ]:
road_edges.dtypes

In [ ]:
road_edges.shape

In [ ]:
road_edges['type'].unique()

In [ ]:
road_edges['form'].unique()

## Exercise: Make IDs Compatible

Our csv files were derived from original [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files, and as you can see from the above, both `road_edges['src_id']` and `road_edges['dst_id']` contain a leading `#` character that `road_nodes['node_id']` does not. To make the IDs compatible between the edges and nodes, use cuDF's [string method](https://docs.rapids.ai/api/nvstrings/stable/) `.str.lstrip` to replace the `src_id` and `dst_id` columns in `road_edges` with values stripped of the leading `#` characters.

### Solution

In [ ]:
%load solutions/make_ids_compatible

## Exercise: Remove Duplicates

Our data originally came from multiple map tiles, and roads which crossed a tile boundary were captured in both the tile from which they started and the tile on which they ended.

Use the `drop_duplicates` method for both `road_nodes` and `road_edges` to remove these duplicate values, and `reset_index` with the parameter `drop=True` to give the new dataframes a contiguous range index.

#### Solution

In [ ]:
%load solutions/remove_duplicates

## Data Summary

Now that the data is cleaned we can see just how many roads and endpoints/junctions/curve points we will be working with, as well as its memory footprint in our GPU.

In [ ]:
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

In [ ]:
!nvidia-smi

## Prep Data for Building the Road Network Graph

The cuGraph graph constructor that we will be using later expects edge sources, edge destinations, and (optionally) edge weights. Edge sources and destinations must be a compact interval of 32-bit integers from 0 to *N*-1, where *N* is the number of nodes in the graph.

Furthermore, in RAPIDS v0.12, undirected edges in cuGraph must be stored as directed edges in both directions. Since our data does not indicate one-way roads, we will assume that all roads can be treated as two-way (undirected).

These constraints will require some preparation, but they allow for the blazing-fast analytics we will be exploring later.

As it stands, and contrary to our needs, our `road_edges` dataframe consists of `src_id` and `dst_id` strings which match `node_id` values in the `road_nodes`. Furthermore, our road edges are recorded as going from one source to one destination even when they are two-way roads.

In [ ]:
road_edges[['src_id', 'dst_id']].dtypes

In [ ]:
road_edges['src_id'][0]

In [ ]:
road_edges['dst_id'][0]

In [ ]:
road_nodes['node_id'].dtype

In [ ]:
road_nodes['node_id'][0]

In order to prepare our data for constructing a graph with cuGraph, we will do the following:

1. Create a unique int32 value for each `node_id`, which we will call that node's `graph_id`, and save the mapping from `node_id` to `graph_id`. Whenever we get results from analyzing the graph, we can use the mapping to look up which original node corresponding to that `graph_id`.
2. Use the mapping to update `road_edges` with `graph_id`s for each edge's start and end points. We will ultimately call the mapped source points `graph_src` and the destination points `graph_dst`.
3. Create a new dataframe with the same edges, but with swapped source and destination points, then concatenate that swapped dataframe to the original and deduplicate (so that each road will be treated in the graph as two-way).

### Create `graph_id`s

The dataframe method `factorize` produces both an integer labeling of the `node_id` values and the mapping series that reflects that labeling.

In [ ]:
all_node_ids = road_edges['src_id'].append(road_edges['dst_id'], ignore_index=True) # ignore_index to prevent index overlap
graph_ids, mapping_series = all_node_ids.factorize()
mapping_series.head()

### Reattach `graph_id`s onto `road_edges`

We now need to add `graph_id` columns into `road_edges` twice: once to create a new source ID column, and again to create a new destination ID column. Because we preserved the order of the `src_id` and `dst_id` columns when sending them to `factorize`, we can simply split the resulting `graph_id`s in half and attach them back to `road_edges`.

In [ ]:
half_index = int((all_node_ids.shape[0]/2))
road_edges['graph_src'] = graph_ids.iloc[:half_index]
road_edges['graph_dst'] = graph_ids.iloc[half_index:].reset_index(drop=True) # reset_index to reverse the ignore_index above
road_edges.head()

### Merge `graph_id` into `road_nodes`

cuDF provides merging functionality just like Pandas. Even though we don't directly need `road_nodes` to construct our graph, we will need it later in the workshop to geospatially analyze where the roads are.

Since we will join the mapping to `road_nodes` via the `node_id` column, we convert the mapping's index to a `graph_id` column.

In [ ]:
node_id_to_graph_id_mapping = cudf.DataFrame()
node_id_to_graph_id_mapping['node_id'] = mapping_series
node_id_to_graph_id_mapping['graph_id'] = cudf.Series(mapping_series.index)
node_id_to_graph_id_mapping.head()

We now add the `graph_id` column to `road_nodes` by performing a merge:

In [ ]:
%time road_nodes = road_nodes.merge(node_id_to_graph_id_mapping, on='node_id', how='left')
road_nodes.head()

#### Reindex `road_nodes`

For efficient lookup later, we will reindex `road_nodes` to use the `graph_id` as its index - remember, we will typically get results from the graph analytics in terms of `graph_id`s, so this lets us easily pull other information about the nodes (like their locations). We then sort the dataframe on this new index:

In [ ]:
road_nodes = road_nodes.set_index('graph_id', drop=True)
%time road_nodes = road_nodes.sort_index()
road_nodes.head()

### Build Dataframe with Columns Needed for Graph Construction

Before making our graph data undirected, let's construct a smaller dataframe with only the columns we will need to give to the cuGraph constructor so we can avoid repeatedly moving unnecessary data:

In [ ]:
graph_prep = cudf.DataFrame()
graph_prep['src']    = road_edges['graph_src']
graph_prep['dst']    = road_edges['graph_dst']
graph_prep['length'] = road_edges['length'].astype('float32') # cuGraph expects edge weights as floating points
print(graph_prep.shape)
graph_prep.dtypes

### Make Graph Undirected

We now make our graph undirected by creating a new dataframe with sources and destinations swapped, concatenating that new dataframe to `graph_prep`:

In [ ]:
rev_gdf = cudf.DataFrame()

rev_gdf['src'] = graph_prep['dst']
rev_gdf['dst'] = graph_prep['src']
rev_gdf['length'] = graph_prep['length']

graph_prep = cudf.concat([graph_prep, rev_gdf], 
                          ignore_index=True)
graph_prep.shape

While it's possible to have road edges with the same starting and ending nodes but different lengths (say, a loop that breaks off from and rejoins a main road), for simplicity we will deduplicate the edge list while ignoring `length` so that there is only one (undirected) edge between each pair of connected nodes.

In [ ]:
graph_prep.drop_duplicates(subset=['src', 'dst'], inplace=True)
graph_prep.shape

Let's confirm `graph_prep` contains a compact interval of int32 IDs:

In [ ]:
# The min src or dst ID is 0
graph_prep[['src', 'dst']].min().min() == 0

In [ ]:
# The max src or dst id is the number of nodes - 1
graph_prep[['src', 'dst']].max().max() == road_nodes['node_id'].unique().shape[0] - 1

In [ ]:
# The ID dtypes are int32s
graph_prep[['src', 'dst']].dtypes

## Construct Graph with cuGraph

Now that we have `graph_prep` prepped correctly, we can use cuGraph to create our graph, which we can then use for accelerated analysis. In order to do so, we first use cuGraph to instantiate a `Graph` instance, and then initialize it with edge sources, edge destinations, and edge weights, which for our data will be the length of the roads:

In [ ]:
G = cg.Graph()
%time G.from_cudf_edgelist(graph_prep, source='src', destination='dst', edge_attr='length')

You can see how fast the whole preparation and construction process can be, even starting from the beginning, by restarting the kernel, clicking on this text, and then selecting "Run All Above Selected Cell" from the Run menu (should take under ten seconds).

Just as a point of comparison, we also construct the equivalent graph in NetworkX from the equivalent cleaned and prepped Pandas dataframe. This should take a minute or so - feel free to read ahead while it processes.

In [ ]:
graph_prep_cpu = graph_prep.to_pandas()

%time G_cpu = nx.convert_matrix.from_pandas_edgelist(graph_prep_cpu, source='src', target='dst', edge_attr='length')

### Analyzing the Graph

Now that we have created the graph we can analyze the number of nodes and edges in it:

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

We can also analyze the degrees of our graph nodes:

In [ ]:
deg_df = G.degree()

We duplicated every edge to make the graph undirected, so we expect the nodes to have a minimum degree of 2:

In [ ]:
deg_df['degree'].describe()[1:]

You will spend more time using this GPU-accelerated graph later in the workshop.

## Exercise: Construct a Graph of Roads with Time Weights

For this series of exercises, you are going to construct and analyze a new graph of Great Britain's roads using the techniques just demonstrated, but this time, instead of using raw distance for the edges' weights, you will be using the time it will take to travel between the two nodes.

You will be beginning this exercise with the `road_edges` dataframe from earlier, which already contains the unique int32 `graph_id`s we need for graph construction in `graph_src` and `graph_dst`:

In [ ]:
road_edges.dtypes

### Road Type to Speed Conversion

In order to calculate how long it should take to travel along a road, we need to know its speed limit. We will do this by utilizing `road_edges['type']`, along with rules for the speed limits for each type of road.

Here are the unique types of roads in our data:

In [ ]:
road_edges['type'].unique()

And here is a table with assumptions about speed limits we can use for our conversion:

In [ ]:
# https://www.rac.co.uk/drive/advice/legal/speed-limits/
# Technically, speed limits depend on whether a road is in a built-up area and the form of carriageway,
# but we can use road type as a proxy for built-up areas.
# Values are in mph.

speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

We begin by creating `speed_gdf` to store each road type with its speed limit:

In [ ]:
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys()
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

Next we add an additional column, `limit_m/s`, which for each road type will give us a measure of how fast one can travel on it in meters / second.

In [ ]:
# We will have road distances in meters (m), so to get road distances in seconds (s), we need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

### Step 1: Merge `speed_gdf` into `road_edges`

Since we will be using values in `road_edges` to construct our graph, we need to merge `speed_gdf` into `road_edges`. You can join on the `type` column, which both of these dataframes share.

#### Solution

In [ ]:
%load solutions/merge_speed_gdf

### Step 2: Add Length in Seconds Column

You now need to calculate the number of seconds it will take to traverse a given road at the speed limit. This can be done by dividing a road's length in m by its speed limit in m/s. Perform this calculation on `road_edges` and store the results in a new column `length_s`.

#### Solution

In [ ]:
%load solutions/length_in_seconds

### Step 3: Construct a Prep Dataframe

Make a new dataframe called `exercise_graph`, add to it three columns: `src`, `dst`, and `length_s`, all which you can obtain from `road_edges`.

#### Solution

In [ ]:
%load solutions/prep_dataframe

### Step 4: Make the Edges Undirected

Make `exercise_graph` undirected by creating a dataframe where `src` and `dst` are reversed, and then concatenating it with `exercise_graph`. Be sure to drop any rows that duplicate other rows' `src` and `dst` values.

#### Solution

In [ ]:
%load solutions/make_undirected

### Step 5: Construct the Graph

Construct a cuGraph Graph called `G_ex` using the sources and destinations found in `exercise_graph`, along with length in seconds values for the edges' weights.

#### Solution

In [ ]:
%load solutions/construct_graph

<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In the next notebook you will work with a dataset representing a population 5 times larger than the UK, a dataset that would not fit in the memory of a single GPU. In order to work with this data you will use Dask cuDF to partition the data among the 4 GPUs at your disposal, and perform the same kinds of data manipulations you have been doing with vanilla cuDF on smaller, single-GPU datasets.